In [ ]:
from __future__ import print_function, division
from quspin.operators import hamiltonian, quantum_operator, exp_op # Hamiltonian and observables
from quspin.basis import spin_basis_1d, spin_basis_general # Hilbert space bases
from quspin.tools.measurements import obs_vs_time, ent_entropy # t_dep measurements
import numpy as np # generic math functions
from numpy.random import ranf,seed # pseudo random numbers
from joblib import delayed,Parallel # parallelisation
from scipy.sparse.linalg import expm
from scipy.optimize import curve_fit
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
import pandas as pd
L = 7
basis = spin_basis_1d(L)
phi_max = np.pi
phi_steps = 128
tmax = 20
tsteps = 200
times = np.linspace(0,tmax, tsteps)
phis = np.linspace(-phi_max, phi_max, phi_steps)
n_iter = 3

def generate_Hamiltonian(v,g,norm = 2,J=1,h=1):
    u = np.sqrt(norm-v**2)
    Jx =  J*((u+v)/2)
    Jy = J*((v-u)/2)
    Jz = J*v
    J_xx = [[Jx, i,i+1] for i in range(L-1)]
    J_yy = [[Jy, i, i+1] for i in range(L-1)]
    J_zz = [[Jz, i, i+1] for i in range(L-1)]
    h_z = np.random.uniform(-h, h, size=L)
    h_x = np.random.uniform(-h, h, size=L)
    disordered_z_field = [[g * h_z[i], i] for i in range(L)]
    disordered_x_field = [[g * h_x[i], i] for i in range(L)]
    static = [["xx",J_xx], ["yy",J_yy], ["zz", J_zz], ["x", disordered_x_field], ["z", disordered_z_field]]
    dynamic = []
    no_checks = {"check_symm": False, "check_herm": False, "check_pcon": False}
    return hamiltonian(static, dynamic, basis = basis, dtype = np.float64, **no_checks)

def W_time(H, W_0):
    """The W_t is the unitary time evolution of either a density matrix
    or an operator that we evolve in the same manner as a density matrix.
    Notice that for the Heisenberg picture of an operator evolved in time is:
    W(t) = U†WU
    and that of a density matrix is:
    p(t) = Up_0U†
    We weill use the second convention for evolving p = Z or p = X in this code,
    following the convention in Krishan thesis. If you desire to evolve an operator
    according to the heisenberg picture, supply the negatives of the times list to
    the evolve method"""

    W_t_backward = H.evolve(W_0, 0, times, eom="SE").T # Results in USz_totalU†
    W_t_forward = H.evolve(W_0, 0, -times, eom="SE").T
    return W_t_backward, W_t_forward

def V_operator(phi, operator):
    V = expm(1.0j * phi * operator)
    V_conj = V.conj() # Sz_total† = Sz_total ---> [exp(i*phi*Sz_total)]† = exp(-i*phi*Sz_total)
    return V, V_conj

# Define the operator or the density matrix to be evolved in time down here
"""For a density matrix:
state = |\psi>
W_0 = state.conj().T @ state"""
# Defining the W_0 = Z
Sz_list = [[0.5, i] for i in range(L)]  # Sz/2 for spin-1/2 system
no_checks = {"check_symm": False, "check_herm": False, "check_pcon": False}
# Building up the Sz_total as hamiltonian and converting it into a matrix
Sz_total = hamiltonian([["z", Sz_list]], [], basis=basis, dtype=np.float64, **no_checks).toarray()
W_0 = Sz_total
def double_quantum_hamiltonian(L, h_x = 1, h_z = 1):
    sigma_x = [[1, i,i+1] for i in range(L-1)]
    sigma_y = [[-1, i,i+1] for i in range(L-1)]
    h_z = np.random.uniform(-h_z, h_z, size=L)
    h_x = np.random.uniform(-h_x, h_x, size=L)
    disordered_z_field = [[h_z[i], i] for i in range(L)]
    disordered_x_field = [[h_x[i], i] for i in range(L)]
    static = [["xx", sigma_x], ["yy", sigma_y], ["x", disordered_x_field], ["z", disordered_z_field]]
    dynamic = []
    no_checks = {"check_symm": False, "check_herm": False, "check_pcon": False}
    return hamiltonian(static, dynamic, basis = basis, dtype = np.float64, **no_checks)


In [ ]:
def calculating_OTOC(V, V_conj, step, W_backward, W_forward):
    # Step is the index of the timestep we are looking at, not the value of t
    # Calculates the OTOC at the timestep = step for the time-dependent operator W_forward and the time-independent rotation operator V
    """Since trace is cyclic, we have: 
    Tr[U† * A *  U * p * U† * A† * U * p] = Tr[U * p * U† * A *  U * p * U† * A†]
    = Tr[p(t)Ap(t)A†]
    p(t) corresponds to W_t here; It's the forward time evolution of the density matrix p. A corresponds to the rotation vector V"""

    W_t_step_forward = W_forward[step]
    W_t_step_backward = W_backward[step]
    OTOC = W_t_step_backward @ V_conj @ W_t_step_forward @ V
    OTOC_value = OTOC.trace().real
    return OTOC_value

#Calculates C(phi, t)
def build_C_phi_t(H, W_0):
    W_t_backward, W_t_forward = W_time(H, W_0)
    C_phi_t = []
    for time_index in range(len(times)):
        C_phi = []
        for phi in phis:
            V, V_conj = V_operator(phi, Sz_total)
            C_phi_value = calculating_OTOC(V, V_conj, time_index, W_t_backward, W_t_forward)
            C_phi.append(C_phi_value)
        C_phi_t.append(C_phi)
    return C_phi_t

In [ ]:
#Calculates C(phi, t)
def build_C_phi_t(H, W_0):
    W_t_backward, W_t_forward = W_time(H, W_0)
    C_phi_t = []
    for time_index in range(len(times)):
        C_phi = []
        for phi in phis:
            V, V_conj = V_operator(phi, Sz_total)
            C_phi_value = calculating_OTOC(V, V_conj, time_index, W_t_backward, W_t_forward)
            C_phi.append(C_phi_value)
        C_phi_t.append(C_phi)
    return C_phi_t

# Build the C(phi) at a specific t. No need to build C(phi, t) for all t; makes the running time faster
def build_C_phi(H, W_0, time_index):
    W_t_backward, W_t_forward = W_time(H, W_0)
    C_phi = []
    for phi in range(len(phis)):
        V, V_conj = V_operator(phis[phi], Sz_total)
        C_phi_value = calculating_OTOC(V, V_conj, time_index, W_t_backward, W_t_forward)
        C_phi.append(C_phi_value)
    return np.array(C_phi)

#Plotting C(phi)
def plot_C_phi(H, W_0, time_index):
    C_phi = build_C_phi(H, W_0, time_index)
    plt.figure(figsize=(8, 6))
    plt.subplot(1, 2, 2)
    plt.plot(phis, C_phi, label=r'$C(\phi)$', linestyle = "-", color='blue')
    #plt.scatter(peak_frequencies, peak_values, color='red', s=20, label='Peaks')
    plt.xlabel(r'$\phi$')
    plt.ylabel(r'$C(\phi)$')
    plt.legend()
    plt.tight_layout()
    plt.show()

#Plotting multiple C(phi) for different double quantum hamiltonians
#Alphas correspond to h_x, betas correspond to h_z
def plot_multiple_C_phi(W_0, time_index, alphas_betas):
    plt.figure(figsize=(10, 7))
    for pair in alphas_betas:
        h_x, h_z = pair[0], pair[1]
        H = double_quantum_hamiltonian(L, h_x, h_z)
        C_phi = build_C_phi(H, W_0, time_index)
        plt.plot(phis, C_phi, linestyle='-', label=f'h_x = {h_x}, h_z = {h_z}')
    plt.xlabel('Time')
    plt.ylabel('OTOC value')
    plt.grid(True)
    plt.legend()  # Add legend to distinguish different Hamiltonians
    plt.show()

#Example
alphas_betas = [[0, 6], [6, 0], [0,0]]
plot_multiple_C_phi(W_0, 40, alphas_betas)

#The following function does the same thing but using hamiltonians generated by generate_hamiltonian
"""
def plot_multiple_C_phi_2(W_0, time_index, vs_gs):
    plt.figure(figsize=(10, 7))
    for pair in vs_gs:
        v, g = pair[0], pair[1]
        H = generate_Hamiltonian(v,g)
        C_phi = build_C_phi(H, W_0, time_index)
        plt.plot(phis, C_phi, linestyle='-', label=f'v = {v}, g = {g}')
    plt.xlabel('Time')
    plt.ylabel('OTOC value')
    plt.grid(True)
    plt.legend()  # Add legend to distinguish different Hamiltonians
    plt.show()
    
#Example
vs_gs = [[0.2,1], [0.2, 2], [0.2, 3], [0.2, 4]]
plot_multiple_C_phi_2(W_0, time_index = 40, vs_gs)
"""



In [ ]:
def calculate_second_moment(frequencies, I_m_list):
    sum = 0
    for j in range(len(frequencies)):
        product = (frequencies[j]**2) * I_m_list[j]
        sum += product
    return sum

"""There are two approaches for building up the fourier coefficients. the following function builds up C(phi, t) inside the function
while the other one takes the C(phi,t) as an argument in case it has been calculated somewhere already in a sequence of calculations so
there's no need to calculate it once more"""
def fourier_transform(H, W_0, time_index):
    C_phi = build_C_phi(H, W_0, time_index)
    I_m = np.fft.fft(C_phi) / phi_steps
    I_m = np.abs(np.real(I_m)/np.sum(np.abs(np.real(I_m))))

    # Get the corresponding m values
    m = np.fft.fftfreq(phi_steps, d=(phis[1] - phis[0]) / (2 * np.pi))
    second_moment = calculate_second_moment(m, I_m)
    return m, I_m, second_moment

def fourier_with_cell(C_phi_t_cell, time_index):
    C_phi = C_phi_t_cell[time_index]
    I_m = np.fft.fft(C_phi) / phi_steps
    I_m = np.abs(np.real(I_m) / np.sum(np.abs(np.real(I_m))))

    # Get the corresponding m values
    m = np.fft.fftfreq(phi_steps, d=(phis[1] - phis[0]) / (2 * np.pi))
    second_moment = calculate_second_moment(m, I_m)
    return m, I_m, second_moment

def plot_I(m, I_m):
    second_moment = calculate_second_moment(m, I_m)
    print(second_moment)
    # Plot the real part of I_m as a function of m
    plt.figure(figsize=(10, 6))
    plt.stem(m, np.real(I_m), use_line_collection=True)
    plt.title('Fourier Coefficients I_m vs m')
    plt.xlabel('m')
    plt.ylabel('Real part of I_m')
    plt.xlim(-20, 20)  # Adjust the range as needed
    plt.grid(True)
    plt.show()

#Example
H_DQ = double_quantum_hamiltonian(L, h_x = 0, h_z = 0 )
m, I_m, second_moment = fourier_transform(H_DQ, W_0, 100)
plot_I(m, I_m)

In [ ]:
# Multiple hamiltonians of interest. Supply the lists of v and g values 
def lineplots(v_s, g_s):
    plt.figure(figsize=(10, 7))
    
    for i in range(len(v_s)):
        v_value = v_s[i]
        g_value = g_s[i]
        averaged_second_moment = []
        for j in range(n_iter):
            hamiltonian = generate_Hamiltonian(v = v_value, g = g_value)
            OTOCS_cell = build_C_phi_t(hamiltonian, W_0)
            second_moment_values = []
            for time_index in range(100):
                freq, abs_C_ftt, value = fourier_with_cell(OTOCS_cell, time_index)
                second_moment_values.append(value)
            if len(averaged_second_moment) == 0:
                averaged_second_moment = np.array(second_moment_values)
            else:
                averaged_second_moment += np.array(second_moment_values)
        averaged_second_moment /= n_iter
        averaged_second_moment = averaged_second_moment.tolist()
        
        # Plot each Hamiltonian's data with a unique color or style
        #second_moment_values[0] = 0
        plt.plot(times[0:100], averaged_second_moment, linestyle='-', label=f'v = {v_value}, g = {g_value}')
    
    #plt.xscale('symlog')
    plt.xlabel('Time')
    plt.ylabel('Second Moment')
    plt.grid(True)
    plt.legend()  # Add legend to distinguish different Hamiltonians
    plt.show()
#Just one hamiltonian. This function is useless in retrospect since we can just about use the above one for one hamiltonian
def lineplot(v_value, g_value, J_value):
    averaged_second_moment = [] # Averaging over n_iterations of disoreder realizations
    for j in range(n_iter):
        hamiltonian = generate_Hamiltonian(v = v_value, g = g_value, J = J_value)
        second_moment_values = [] # Second moment values for a specifc iteration
        for time_index in range(1000):
            freq, abs_C_ftt, value = fourier_transform(hamiltonian, W_0, time_index)
            second_moment_values.append(value)

        if len(averaged_second_moment) == 0:
            averaged_second_moment = np.array(second_moment_values)
        else:
            # Adding the values to the main averaged list and then dividing by the number of iterations at the end
            averaged_second_moment += np.array(second_moment_values) 

    averaged_second_moment /= n_iter
    averaged_second_moment = averaged_second_moment.tolist()
    averaged_second_moment[0] = 0
        
    plt.figure(figsize=(8, 6))
    plt.plot(times[0:100], averaged_second_moment, marker='o', markersize = 3,linestyle='-', color='b')
    #plt.xscale('symlog')  # Set x-axis to logarithmic scale
    plt.xlabel('Time')
    plt.ylabel('Second Moment')
    #plt.yscale('log')
    plt.grid(True)
    plt.show()

# And for a general hamiltonian that can be supplied
def lineplot_general(hamiltonian):
    
    averaged_second_moment = [] # Averaging over n_iterations of disoreder realizations
    for j in range(n_iter):
        
        second_moment_values = [] # Second moment values for a specifc iteration
        for time_index in range(0, 100):
            freq, abs_C_ftt, value = fourier_transform(hamiltonian, W_0, time_index)
            second_moment_values.append(value)

        if len(averaged_second_moment) == 0:
            averaged_second_moment = np.array(second_moment_values)
        else:
            # Adding the values to the main averaged list and then dividing by the number of iterations at the end
            averaged_second_moment += np.array(second_moment_values) 

    averaged_second_moment /= n_iter
    averaged_second_moment = averaged_second_moment.tolist()
    averaged_second_moment[0] = 0
        
    plt.figure(figsize=(8, 6))
    plt.plot(times[0:100], averaged_second_moment, marker='o', markersize = 3,linestyle='-', color='b')
    #plt.xscale('symlog')  # Set x-axis to logarithmic scale
    plt.xlabel('Time')
    plt.ylabel('Second Moment')
    #plt.yscale('log')
    plt.grid(True)
    plt.show()

#Example 1
v_s = [0.9,0.9,0.9,0.9,0.9]
g_s = [0,1,2,3,4]
lineplots(v_s,g_s)
#Example 2
lineplot(0.5,1, J_value = 1)
#Example 3
H_DQ = double_quantum_hamiltonian(L, h_x = 0, h_z = 0)
lineplot_general(H_DQ)




